### Installations


In [2]:
!uv pip install numba

Using Python 3.12.9 environment at: /Users/swap357/Documents/dev/learning-numba/.venv
Resolved 3 packages in 99ms                                          
Installed 3 packages in 49ms                                
 + llvmlite==0.44.0
 + numba==0.61.0
 + numpy==2.1.3


## objective: investigate np.clip and np.kron issues

This notebook demos numba's implementation of np.clip and np.kron compared to numpy and possible solution.

#### np.clip issues:
   - Different behavior with NaN/Inf bounds ([#9995](https://github.com/numba/numba/issues/9995), [#10000](https://github.com/numba/numba/issues/10000))
   - Type promotion issues with NaN/inf values
   - Different behaviour with broadcasting ([#9991](https://github.com/numba/numba/issues/9991))


#### np.kron issues:
   - Type promotion differences - boolean array ([#9992](https://github.com/numba/numba/issues/9992))



In [2]:
import numpy as np
from numba import njit

In [3]:
import numba
print(f"NumPy version: {np.__version__}")
print(f"Numba version: {numba.__version__}")

NumPy version: 2.1.3
Numba version: 0.61.0


### np.clip behavior for normal use-case

In [4]:
arr = np.array([1, 2, 5, 10, 15])
a_min = np.array([3])
a_max = np.array([12])
print("Input arrays:")
print("arr.dtype:", arr.dtype)
print("a_min.dtype:", a_min.dtype)
print("a_max.dtype:", a_max.dtype)
print(" --------------- ")

Input arrays:
arr.dtype: int64
a_min.dtype: int64
a_max.dtype: int64
 --------------- 


#### checking expected result type when obeying numpy promotion rules

In [8]:
common_dtype = np.result_type(arr, a_min, a_max)
print("np.result_type(arr, a_min, a_max):", common_dtype)

np.result_type(arr, a_min, a_max): int64


In [10]:
py_result = np.clip(arr, a_min, a_max)
print("np.clip(arr, a_min, a_max):", py_result)
print("py_result.dtype:", py_result.dtype)
print(" --------------- ")

np.clip(arr, a_min, a_max): [ 3  3  5 10 12]
py_result.dtype: int64
 --------------- 


In [11]:
@njit
def clip_njit(arr, a_min, a_max):
    return np.clip(arr, a_min, a_max)
nb_result = clip_njit(arr, a_min, a_max)
print("@numba.njit np.clip(arr, a_min, a_max):", nb_result)
print("nb_result.dtype:", nb_result.dtype)

@numba.njit np.clip(arr, a_min, a_max): [ 3  3  5 10 12]
nb_result.dtype: int64


### np.clip behavior for when either of a_min or a_max = np.nan

In [12]:
arr = np.array([1, 2, 5, 10, 15])
a_min = np.nan
a_max = 12
print("Input arrays:")
print("arr.dtype:", arr.dtype)
print("a_min:", a_min)
print("a_max:", a_max)

Input arrays:
arr.dtype: int64
a_min: nan
a_max: 12


In [14]:
common_dtype = np.result_type(arr, a_min, a_max)
print("np.result_type(arr, a_min, a_max):", common_dtype)

np.result_type(arr, a_min, a_max): float64


In [15]:
result = np.clip(arr, a_min, a_max)
print("np.clip(arr, a_min, a_max):", result)
print("result.dtype:", result.dtype)

np.clip(arr, a_min, a_max): [nan nan nan nan nan]
result.dtype: float64


In [17]:
@njit
def clip_njit_nan(arr, a_min, a_max):
    return np.clip(arr, a_min, a_max)
result = clip_njit_nan(arr, a_min, a_max)
print("@numba.njit np.clip(arr, a_min, a_max):", result)
print("result.dtype:", result.dtype)

@numba.njit np.clip(arr, a_min, a_max): [ 1  2  5 10 12]
result.dtype: int64


numba’s np.clip does not handle NaN bounds the same way as np does

#### on numpy
NOTE: type promotion on arrays: (https://github.com/numpy/numpy/blob/9389862162bbd46b5324402d6a08a27bc18ddb7d/doc/source/reference/arrays.promotion.rst)
- promotion rules as defined by NEP 50: (https://numpy.org/neps/nep-0050-scalar-promotion.html) 

- when np.clip is called, python implementation delgates it to array method via _wrapfunc, implemented in C.
- https://github.com/numpy/numpy/blob/9389862162bbd46b5324402d6a08a27bc18ddb7d/numpy/_core/fromnumeric.py#L2242
- https://github.com/numpy/numpy/blob/9389862162bbd46b5324402d6a08a27bc18ddb7d/numpy/_core/src/multiarray/calculation.c#L790C1-L790C13
- https://github.com/numpy/numpy/blob/9389862162bbd46b5324402d6a08a27bc18ddb7d/numpy/_core/src/multiarray/convert_datatype.c#L1573

#### on numba
- https://github.com/numba/numba/blob/c21aa9273ef4298392695b1f4613d29456b53e5c/numba/np/arrayobj.py#L2342





In [18]:
# _np_clip_impl(a, a_min, a_max, out) np.clip helper function on numba
# https://github.com/numba/numba/blob/c21aa9273ef4298392695b1f4613d29456b53e5c/numba/np/arrayobj.py#L2342
def numba_np_clip(a, a_min, a_max):
    ret = np.empty_like(a)
    a_b, a_min_b, a_max_b = np.broadcast_arrays(a, a_min, a_max)
    for index in np.ndindex(a_b.shape):
        val_a = a_b[index]
        val_a_min = a_min_b[index]
        val_a_max = a_max_b[index]
        ret[index] = min(max(val_a, val_a_min), val_a_max)

    return ret

In [30]:
arr = np.array([1, 2, 5, 10, 15])
a_min = np.nan
a_max = np.nan
print("Input arrays:")
print("arr.dtype:", arr.dtype)
print("a_min:", a_min)
print("a_max:", a_max)
common_dtype = np.result_type(arr, a_min, a_max)
print("np.result_type(arr, a_min, a_max):", common_dtype)

@njit
def clip_njit(arr, a_min, a_max):
    return np.clip(arr, a_min, a_max)

nb_result = clip_njit(arr, a_min, a_max)
py_result = clip_njit.py_func(arr, a_min, a_max)
print()
print("numba result:", nb_result)
print("numba result dtype:", nb_result.dtype)
print("--------------------------------")
print("python result:", py_result)
print("python result dtype:", py_result.dtype)

Input arrays:
arr.dtype: int64
a_min: nan
a_max: nan
np.result_type(arr, a_min, a_max): float64

numba result: [ 1  2  5 10 15]
numba result dtype: int64
--------------------------------
python result: [nan nan nan nan nan]
python result dtype: float64


In [33]:
arr = np.array([1, 2, 5, 10, 15])
a_min = np.inf
a_max = 12
print("Input arrays:")
print("arr.dtype:", arr.dtype)
print("a_min:", a_min)
print("a_max:", a_max)
common_dtype = np.result_type(arr, a_min, a_max)
print("np.result_type(arr, a_min, a_max):", common_dtype)
nb_result = clip_njit(arr, a_min, a_max)
py_result = clip_njit.py_func(arr, a_min, a_max)
print()
print("numba result:", nb_result)
print("numba result dtype:", nb_result.dtype)
print("--------------------------------")
print("python result:", py_result)
print("python result dtype:", py_result.dtype)

Input arrays:
arr.dtype: int64
a_min: inf
a_max: 12
np.result_type(arr, a_min, a_max): float64

numba result: [12 12 12 12 12]
numba result dtype: int64
--------------------------------
python result: [12. 12. 12. 12. 12.]
python result dtype: float64


In [34]:
arr = np.array([1, 2, 5, 10, 15])
a_min = 3
a_max = np.inf
print("Input arrays:")
print("arr.dtype:", arr.dtype)
print("a_min:", a_min)
print("a_max:", a_max)
common_dtype = np.result_type(arr, a_min, a_max)
print("np.result_type(arr, a_min, a_max):", common_dtype)
nb_result = clip_njit(arr, a_min, a_max)
py_result = clip_njit.py_func(arr, a_min, a_max)
print()
print("numba result:", nb_result)
print("numba result dtype:", nb_result.dtype)
print("--------------------------------")
print("python result:", py_result)
print("python result dtype:", py_result.dtype)

Input arrays:
arr.dtype: int64
a_min: 3
a_max: inf
np.result_type(arr, a_min, a_max): float64

numba result: [ 3  3  5 10 15]
numba result dtype: int64
--------------------------------
python result: [ 3.  3.  5. 10. 15.]
python result dtype: float64


### revise np_clip implementation on numba with:

 type promotion for nan and inf (float64) : to address #9995 and #10000
```
    dtype = np.result_type(a, a_min, a_max)
    ret = np.empty(br_shape, dtype=dtype)
```
 
 and output shape accounting shapes of a_min and a_max as well : to address #9991
```
    br_shape = np.broadcast(a, a_min, a_max).shape
    ret = np.empty(br_shape, dtype=dtype)
```

existing implementation on numba:
https://github.com/numba/numba/blob/c21aa9273ef4298392695b1f4613d29456b53e5c/numba/np/arrayobj.py#L2342

In [50]:
def revised_np_clip_impl(a, a_min, a_max):
    dtype = np.result_type(a, a_min, a_max)
    br_shape = np.broadcast(a, a_min, a_max).shape
    ret = np.empty(br_shape, dtype=dtype)
    a_b, a_min_b, a_max_b = np.broadcast_arrays(a, a_min, a_max)
    for index in np.ndindex(a_b.shape):
        val_a = a_b[index]
        val_a_min = a_min_b[index]
        val_a_max = a_max_b[index]
        # Propagate NaN if either bound is NaN.
        if np.isnan(val_a_min) or np.isnan(val_a_max):
            ret[index] = np.nan
        else:
            ret[index] = min(max(val_a, val_a_min), val_a_max)

    return ret

In [51]:
print("arr:", arr)
print()
a_min = 1
a_max = 2
print("* clip(arr, a_min=1, a_max=2) :")

revised_impl_result = revised_np_clip_impl(arr, a_min, a_max)
print("revised_impl:", revised_impl_result)
print("revised_impl.dtype:", revised_impl_result.dtype)

py_result = np.clip(arr, a_min, a_max)
print("numpy:", py_result)
print("numpy.dtype:", py_result.dtype)
print("--------------------------------")
a_min = np.nan
a_max = np.nan
print("* clip(arr, a_min=np.nan, a_max=np.nan) :")

revised_impl_result = revised_np_clip_impl(arr, a_min, a_max)
print("revised_impl:", revised_impl_result)
print("revised_impl.dtype:", revised_impl_result.dtype)

py_result = np.clip(arr, a_min, a_max)
print("numpy:", py_result)
print("numpy.dtype:", py_result.dtype)
print("--------------------------------")

arr: [ 1  2  5 10 15]

* clip(arr, a_min=1, a_max=2) :
revised_impl: [1 2 2 2 2]
revised_impl.dtype: int64
numpy: [1 2 2 2 2]
numpy.dtype: int64
--------------------------------
* clip(arr, a_min=np.nan, a_max=np.nan) :
revised_impl: [nan nan nan nan nan]
revised_impl.dtype: float64
numpy: [nan nan nan nan nan]
numpy.dtype: float64
--------------------------------


In [54]:
a_min = np.inf
a_max = 12
print("arr:", arr)
print()

print("* clip(arr, a_min=1, a_max=2) :")
print("revised:", revised_np_clip_impl(arr, a_min, a_max))
print("numpy:", np.clip(arr, a_min, a_max))

a_min = 3
a_max = np.inf
print("--------------------------------")
print("* clip(arr, a_min=np.nan, a_max=np.nan) :")
print("revised:", revised_np_clip_impl(arr, a_min, a_max))
print("numpy:", np.clip(arr, a_min, a_max))

arr: [ 1  2  5 10 15]

* clip(arr, a_min=1, a_max=2) :
revised: [12. 12. 12. 12. 12.]
numpy: [12. 12. 12. 12. 12.]
--------------------------------
* clip(arr, a_min=np.nan, a_max=np.nan) :
revised: [ 3.  3.  5. 10. 15.]
numpy: [ 3.  3.  5. 10. 15.]


### addressing #9991: https://github.com/numba/numba/issues/9991

In [57]:
arr = np.array([[1], [2]])
a_min = 0
a_max = np.array([3, 4, 5])

@njit
def clip_njit(arr, a_min, a_max):
    return np.clip(arr, a_min, a_max)

nb_result = clip_njit(arr, a_min, a_max)
py_result = clip_njit.py_func(arr, a_min, a_max)
print("numba result:", nb_result)
print("numba result shape:", nb_result.shape)
print("--------------------------------")
print("python result:", py_result)
print("python result shape:", py_result.shape)


numba result: [[1]
 [2]]
numba result shape: (2, 1)
--------------------------------
python result: [[1 1 1]
 [2 2 2]]
python result shape: (2, 3)


In [60]:
revised_impl_result = revised_np_clip_impl(np.array(arr), a_min, a_max)
print("revised_impl:", revised_impl_result)
print("revised_impl shape:", revised_impl_result.shape)
print("--------------------------------")


revised_impl: [[1 1 1]
 [2 2 2]]
revised_impl shape: (2, 3)
--------------------------------


## np.kron issue: [#9992](https://github.com/numba/numba/issues/9992)
   - Type promotion differences between numpy and numba
   - Boolean array handling

In [62]:
a = np.array([True])
b = np.array([5])
print("a.dtype:", a.dtype)
print("b.dtype:", b.dtype)
common_dtype = np.result_type(a, b)
print("np.result_type(a, b):", common_dtype)

a.dtype: bool
b.dtype: int64
np.result_type(a, b): int64


In [ ]:
a = np.array([True])
b = np.array([5])
print("a.dtype:", a.dtype)
print("b.dtype:", b.dtype)
common_dtype = np.result_type(a, b)
promoted_dtype = np.promote_types(a.dtype, b.dtype)
print("np.result_type(a, b):", common_dtype)
print("--------------------------------")
result = np.kron(a, b)
print("np.kron(a, b):", result)
print("result.dtype:", result.dtype)
print("--------------------------------")
@njit
def kron_njit(a, b):
    return np.kron(a, b)
result = kron_njit(a, b)
print("@numba.njit np.kron(a, b):", result)
print("result.dtype:", result.dtype)


a.dtype: bool
b.dtype: int64
np.result_type(a, b): int64
np.promote_types(a.dtype, b.dtype): int64
--------------------------------
np.kron(a, b): [5]
result.dtype: int64
--------------------------------
@numba.njit np.kron(a, b): [ True]
result.dtype: bool
